In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.geocoders import Nominatim

Zunächst wird die Excel Datei, welche von dem Crawler erstellt wird, in ein Dataframe geladen.

In [2]:
df = pd.read_excel ('./output_transfermarkt.xlsx')
df.head(10)

,Unnamed: 0,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge
0,0,Ederson,"56,00 Mio. €",31,17.08.1993 (26),Manchester City,\n\t\t\t\t\t\t\t\tTorwart ...,"1,88 m",\nOsasco (SP)\n,Premier League
1,1,Claudio Bravo,"1,20 Mio. €",1,13.04.1983 (37),Manchester City,\n\t\t\t\t\t\t\t\tTorwart ...,"1,84 m",\nViluco\n,Premier League
2,2,John Stones,"36,00 Mio. €",5,28.05.1994 (26),Manchester City,\n\t\t\t\t\t\t\t\tInnenverteidiger ...,"1,88 m",\nBarnsley\n,Premier League
3,3,Eric García,"6,30 Mio. €",50,09.01.2001 (19),Manchester City,\n\t\t\t\t\t\t\t\tInnenverteidiger ...,"1,83 m",\nBarcelona\n,Premier League
4,4,Joel Latibeaudiere,100 Tsd. €,-,06.01.2000 (20),Man City U23,\n\t\t\t\t\t\t\t\tInnenverteidiger ...,"1,80 m",\nDoncaster\n,Premier League
5,5,Oleksandr Zinchenko,"24,00 Mio. €",11,15.12.1996 (23),Manchester City,\n\t\t\t\t\t\t\t\tLinker Verteidiger ...,"1,75 m","\nRadomyshl, Zhytomyr ...\n",Premier League
6,6,João Cancelo,"36,00 Mio. €",27,27.05.1994 (26),Manchester City,\n\t\t\t\t\t\t\t\tRechter Verteidiger ...,"1,82 m",\nBarreiro\n,Premier League
7,7,Fernandinho,"8,00 Mio. €",25,04.05.1985 (35),Manchester City,\n\t\t\t\t\t\t\t\tDefensives Mittelfeld ...,"1,79 m",\nLondrina\n,Premier League
8,8,Phil Foden,"27,00 Mio. €",47,28.05.2000 (20),Manchester City,\n\t\t\t\t\t\t\t\tZentrales Mittelfeld ...,"1,71 m",\nStockport\n,Premier League
9,9,David Silva,"12,00 Mio. €",21,08.01.1986 (34),Manchester City,\n\t\t\t\t\t\t\t\tOffensives Mittelfeld ...,"1,70 m",\nArguineguín\n,Premier League


Nun werden die Daten "bereiningt".

Zunächst werden alle Textsteuerelemnte  welche noch in den Zeilen vorhanden sind rausgelöscht, sodass das Dataframe hiervon befreit ist. 

In [3]:
df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
df.head(3)

,Unnamed: 0,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge
0,0,Ederson,"56,00 Mio. €",31,17.08.1993 (26),Manchester City,Torwart,"1,88 m",Osasco (SP),Premier League
1,1,Claudio Bravo,"1,20 Mio. €",1,13.04.1983 (37),Manchester City,Torwart,"1,84 m",Viluco,Premier League
2,2,John Stones,"36,00 Mio. €",5,28.05.1994 (26),Manchester City,Innenverteidiger,"1,88 m",Barnsley,Premier League


Nun wird die Spalte Unnamed:0 gelöscht da diese in der späteren Analyse nicht benötigt wird.

In [4]:
del df['Unnamed: 0']
df.head(3)

,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge
0,Ederson,"56,00 Mio. €",31,17.08.1993 (26),Manchester City,Torwart,"1,88 m",Osasco (SP),Premier League
1,Claudio Bravo,"1,20 Mio. €",1,13.04.1983 (37),Manchester City,Torwart,"1,84 m",Viluco,Premier League
2,John Stones,"36,00 Mio. €",5,28.05.1994 (26),Manchester City,Innenverteidiger,"1,88 m",Barnsley,Premier League


Nun wird das in den klammern stehende Alter in der BDay Spalte gelöscht und der Geburtstag in ein Datumsformat umgewandelt. Sodass hiermit spätere Rechnungen durchgeführt werdeb kann. Voher werden allerdings erst die Spieler gelöscht wo die Datumswert nicht vohrhanden sind um eine Grundgesamtheit zu haben, die alle Informationen beinhaltet. Da jedeglich die Spieler kein Geburtstadatum haben, welche nicht in einer Europäischen Profi Liga spielen, sind diese in der späterer Analyse nicht von bedeutung.

In [5]:
df['BDay'] = df['BDay'].str.replace(r' .+', '')
df['BDay'] = df['BDay'].str.replace('.', '')
df.drop(df.loc[df['BDay']=='-'].index, inplace=True)
df.drop(df.loc[df['BDay']=='k'].index, inplace=True)
df['BDay'] = pd.to_datetime(df['BDay'], format='%d%m%Y')
df.head(3)

,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge
0,Ederson,"56,00 Mio. €",31,1993-08-17,Manchester City,Torwart,"1,88 m",Osasco (SP),Premier League
1,Claudio Bravo,"1,20 Mio. €",1,1983-04-13,Manchester City,Torwart,"1,84 m",Viluco,Premier League
2,John Stones,"36,00 Mio. €",5,1994-05-28,Manchester City,Innenverteidiger,"1,88 m",Barnsley,Premier League


In [6]:
today = datetime.now()
print(today)
AgeList = []
i = 0
for index, row in df.iterrows():
    age = today - row['BDay']
    age = round(age.days / 365)
    AgeList.append(age)
df['Age'] = AgeList
df.head(3)

2020-08-19 08:45:01.260896


,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge,Age
0,Ederson,"56,00 Mio. €",31,1993-08-17,Manchester City,Torwart,"1,88 m",Osasco (SP),Premier League,27
1,Claudio Bravo,"1,20 Mio. €",1,1983-04-13,Manchester City,Torwart,"1,84 m",Viluco,Premier League,37
2,John Stones,"36,00 Mio. €",5,1994-05-28,Manchester City,Innenverteidiger,"1,88 m",Barnsley,Premier League,26


In [7]:
gc = Nominatim(user_agent='myGeocoder')

LatList = []
LonList = []
CountryList = []
for index, row in df.iterrows():
    try:
        country = gc.geocode(row["Birth Place"])
        lat = country.raw['lat']
        lon = country.raw['lon']
        country = gc.reverse([lat, lon])
        country = country.raw['address']['country']
        print(country)
    except:
        lat = "-"
        lon = "-"
        country = "-"
    LatList.append(lat)
    LonList.append(lon)
    CountryList.append(country)

df['Lon'] = LonList
df['Lat'] = LatList
df['country'] = CountryList
df.head(3)


Brasil
Chile
United Kingdom
España
United Kingdom
Україна
Portugal
Brasil
United Kingdom
España
Jamaika
Portugal
United Kingdom
Argentina
Switzerland
France
Argentina
United Kingdom
France
United Kingdom
España
Deutschland
België - Belgique - Belgien
日本 (Japan)
Deutschland
France
Brasil
France
Nederland
Colombia
België - Belgique - Belgien
United Kingdom
United Kingdom
United Kingdom
United Kingdom
Argentina
France
United Kingdom
대한민국
Brasil
United Kingdom
United States of America
United Kingdom
België - Belgique - Belgien
Bolivia
Australia
Côte d'Ivoire
United Kingdom
France
United Kingdom
São Tomé e Príncipe
United Kingdom
Nederland
Argentina
United Kingdom
España
United Kingdom
United Kingdom
Côte d'Ivoire
United Kingdom
United Kingdom
United Kingdom
Nederland
France
Brasil
België - Belgique - Belgien
United Kingdom
United Kingdom
France
Nigeria
Argentina
Switzerland
Sverige
République Démocratique du Congo
United Kingdom
United Kingdom
Portugal
Србија (Serbia)
United States of Amer

Italia
Italia
Italia
Italia
Brasil
Italia
Österreich
Italia
España
Côte d'Ivoire
France
Italia
Italia
Italia
Italia
Brasil
România
Italia
Ελλάδα
Italia
Deutschland
Italia
France
Србија (Serbia)
Slovensko
France
Italia
Polska
Sénégal
Nederland
Italia
Nederland
Sénégal
Argentina
Sverige
Italia
Deutschland
Danmark
Paraguay
Brasil
Canada
Italia
Brasil
日本 (Japan)
Chile
Deutschland
Nederland
Česká republika
Italia
The Gambia
Argentina
Italia
Italia
Kosova
Polska
Brasil
Italia
Italia
Italia
Italia
Ghana
Србија (Serbia)
France
Italia
Italia
Србија (Serbia)
Italia
Deutschland
Italia
Italia
Italia
Portugal
Nederland
Italia
Brasil
Italia
Italia
Polska
Italia
Argentina
Italia
Brasil
United States of America
France
Danmark
Italia
Bosna i Hercegovina
Sverige
Italia
Македонија
Italia
Brasil
República de El Salvador
Nederland
Österreich
Nederland
Nederland
República de El Salvador
France
Brasil
Italia
Polska
Kosova
Sénégal
Italia
Brasil
Polska
Italia
Italia
Italia
Bolivia
France
Eesti
Italia
Italia
It

Россия
Россия
Slovenija
Россия
Kyrgyzstan
Hrvatska
Россия
Беларусь
Россия
Deutschland
Россия
Côte d'Ivoire
Россия
Brasil
Hrvatska
Россия
Россия
Россия
Ísland
Россия
Mali
Россия
Россия
Россия
Россия
Россия
Россия
Slovensko
Россия
Zambia
Deutschland
Россия
Россия
Argentina
Россия
France
Россия
Brasil
Россия
Россия
Россия
Россия
United States of America
Россия
Россия
Nederland
Россия
Россия
Hrvatska
Россия
Sverige
Россия
Россия
Россия
Россия
Россия
Россия
Kazakhstan
Россия
Россия
Brasil
Bosna i Hercegovina
Россия
Россия
Россия
Россия
Danmark
Georgia / საქართველო
Switzerland
Georgia / საქართველო
Россия
Россия
Россия
Россия
Россия
Россия
Россия
Россия
Россия
Россия
Georgia / საქართველო
Россия
Shqipëria
Россия
Sénégal
Россия
Venezuela
Hrvatska
Србија (Serbia)
Россия
Україна
Brasil
Eesti
Brasil
Kosova
Россия
Brasil
Deutschland
Россия
Slovenija
Россия
Србија (Serbia)
Россия
Россия
Россия
Ecuador
Armenia
Россия
Россия
Latvija
Georgia / საქართველო
Россия
France
Россия
Россия
Россия
België - Belg

Suomi
Brasil
België - Belgique - Belgien
België - Belgique - Belgien
Norge
Norge
Ghana
Sverige
Nigeria
Türkiye
Hrvatska
Nederland
Sénégal
France
République Démocratique du Congo
Portugal
België - Belgique - Belgien
Cameroon;Cameroun
België - Belgique - Belgien
France
België - Belgique - Belgien
België - Belgique - Belgien
Portugal
België - Belgique - Belgien
België - Belgique - Belgien
België - Belgique - Belgien
België - Belgique - Belgien
België - Belgique - Belgien
België - Belgique - Belgien
Brasil
Sverige
Norge
België - Belgique - Belgien
Ghana
België - Belgique - Belgien
België - Belgique - Belgien
日本 (Japan)
ישראל
Cameroon;Cameroun
France
België - Belgique - Belgien
République Démocratique du Congo
Deutschland
België - Belgique - Belgien
France
Ghana
Ελλάδα
Maroc
France
France
België - Belgique - Belgien
France
Norge
Burundi
Cameroon;Cameroun
België - Belgique - Belgien
België - Belgique - Belgien
België - Belgique - Belgien
Sverige
België - Belgique - Belgien
België - Belgique 

Österreich
Österreich
Deutschland
Cameroon;Cameroun
Österreich
Österreich
Österreich
Österreich
Österreich
Cameroon;Cameroun
Österreich
Bénin
Tanzania
Österreich
Mali
Česká republika
Bosna i Hercegovina
Österreich
Österreich
Österreich
Österreich
Österreich
Österreich
Österreich
Österreich
Österreich
Österreich
Slovenija
Österreich
Österreich
Deutschland
Italia
Brasil
Peru
Ghana
Italia
Österreich
Österreich
Österreich
Brasil
Österreich
Hrvatska
Bosna i Hercegovina
Österreich
Österreich
Österreich
Österreich
Österreich
Österreich
Österreich
Deutschland
Österreich
Italia
Österreich
Ghana
Österreich
United Kingdom
United Kingdom
Norge
United Kingdom
République Démocratique du Congo
United Kingdom
United Kingdom
France
United Kingdom
United Kingdom
United Kingdom
United Kingdom
Україна
Polska
Côte d'Ivoire
United Kingdom
France
Hrvatska
ישראל
United Kingdom
Nederland
ישראל
United Kingdom
United Kingdom
United Kingdom
Ireland
Australia
ایران
France
United Kingdom
Ireland
Česká republika
Uni

...

Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deut

Österreich
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
France
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
United States of America
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Switzerland
Deutschland
France
Deutschland
Deutschland
Italia
Deutschland
Deutschland
Deutschland
Crna Gora
Deutschland
Deutschland
Danmark
Slovenija
Deutschland
Nederland
Deutschland
Deutschland
Canada
Deutschland
Deutschland
Deutschland
Danmark
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
France
Deutschland
Deutschland
Italia
Deutschland
Deutschland
Deutschland
Crna Gora
Deutschland
Deutschland
Danmark
Slovenija
Deutschland
Nederland
Deutschland
Deutschland
Canada
Deutschland
Deutschland
Deutschland
Danmark
Deutschland
Deutschland
Deutschland
Deutsch

Deutschland
Namibia
Deutschland
Deutschland
Deutschland
Ísland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Switzerland
Deutschland
Deutschland
Deutschland
Deutschland
France
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Lietuva
Deutschland
Deutschland
Canada
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Namibia
Deutschland
Deutschland
Deutschland
Ísland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Switzerland
Deutschland
Deutschland
Deutschland
Deutschland
France
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Lietuva
Deutschland
Deutschland
Canada
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Namibia
Deutschland
Deutschland
Deutschland
Ísland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Switzerland
Deutschland
Deutschland
Deutschland
Deutschland
France
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Lietuva
Deutschland
Deutschl

Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Hrvatska
Hrvatska
Deutschland
Angola
Deutschland
República Dominicana
Deutschland
Guinée
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Nederland
Deutschland
Deutschland
Deutschland
Deutschland
Deutschland
Kosova
대한민국


,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge,Age,Lon,Lat,country
0,Ederson,"56,00 Mio. €",31,1993-08-17,Manchester City,Torwart,"1,88 m",Osasco (SP),Premier League,27,-46.7916801,-23.5324859,Brasil
1,Claudio Bravo,"1,20 Mio. €",1,1983-04-13,Manchester City,Torwart,"1,84 m",Viluco,Premier League,37,-71.1593473,-34.8962818,Chile
2,John Stones,"36,00 Mio. €",5,1994-05-28,Manchester City,Innenverteidiger,"1,88 m",Barnsley,Premier League,26,-1.4827755,53.5527719,United Kingdom


Nun müssen noch die aus der Gesamtenmenge herausgenommen werden, bei denen der Geburtsort so wie Land nicht ermittelbar ist.

In [13]:
df.drop(df.loc[df['Lon']=='-'].index, inplace=True)
df.head(142)

,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge,Age,Lon,Lat,country
0,Ederson,"56,00 Mio. €",31,1993-08-17,Manchester City,Torwart,"1,88 m",Osasco (SP),Premier League,27,-46.7916801,-23.5324859,Brasil
1,Claudio Bravo,"1,20 Mio. €",1,1983-04-13,Manchester City,Torwart,"1,84 m",Viluco,Premier League,37,-71.1593473,-34.8962818,Chile
2,John Stones,"36,00 Mio. €",5,1994-05-28,Manchester City,Innenverteidiger,"1,88 m",Barnsley,Premier League,26,-1.4827755,53.5527719,United Kingdom
3,Eric García,"6,30 Mio. €",50,2001-01-09,Manchester City,Innenverteidiger,"1,83 m",Barcelona,Premier League,20,2.1774322,41.3828939,España
4,Joel Latibeaudiere,100 Tsd. €,-,2000-01-06,Man City U23,Innenverteidiger,"1,80 m",Doncaster,Premier League,21,-1.1335312,53.5227681,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Bruno Jordão,"2,30 Mio. €",6,1998-10-12,Wolverhampton,Zentrales Mittelfeld,"1,80 m",Marinha Grande,Premier League,22,-8.961726480980637,39.78962145,Portugal
138,Ming-Yang Yang,250 Tsd. €,39,1995-07-11,Wolverhampton,Zentrales Mittelfeld,"1,72 m",Basel,Premier League,25,7.5878261,47.5581077,Switzerland
139,Daniel Podence,"12,00 Mio. €",10,1995-10-21,Wolverhampton,Linksaußen,"1,65 m",Oeiras,Premier League,25,-9.271300382407272,38.7124971,Portugal
140,Adama Traoré,"25,50 Mio. €",37,1996-01-25,Wolverhampton,Rechtsaußen,"1,78 m",L'Hospitalet de ...,Premier League,25,2.0997927,41.3598601,España


Zueletz, müssen noch die Martkwärte in Zahlen umgewandelt werden damit hieraus z.B. der Durchschnitt berechnet werden kann.

In [3]:
df['Values'] = df['Values'].str.replace(r' Mio.+', '0000')
df['Values'] = df['Values'].str.replace(r' Tsd.+', '000')
df['Values'] = df['Values'].str.replace(',', '')

df['Values'].replace('\s+', np.nan,regex=True,inplace=True)
df['Values'].replace('-', np.nan, inplace=True)
df['Values'].replace('', np.nan, inplace=True)

df.dropna(subset=['Values'], inplace=True)
df['Values'].astype('float')
df

NameError: name 'df' is not defined

Nun werden noch die falsch zugeordneten Daten welche sich in der Spalte Size anstatt ind der Spalte Position befinden neu zugeordnet. Da bei allen Eintragen , bei welchen eine Falsche Zuordnung besteht anstelle der Position das ende des Vertags steht kann man in der Spalte Position nach der Zahl 3  suchen da alle Verträge am Ende des Monates Juli oder August Enden.

In [88]:
for index, row in df.iterrows():
    if '3' in row['Position'] :
        print(row['Position'])
        df.at[index , 'Position'] = row['Size']
df  

30.06.2020
30.06.2020
30.06.2020


,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge,Age,Lon,Lat,country
0,Ederson,5600000000,31,1993-08-17,Manchester City,Torwart,"1,88 m",Osasco (SP),Premier League,27,-46.791680,-23.532486,Brasil
1,Claudio Bravo,120000000,1,1983-04-13,Manchester City,Torwart,"1,84 m",Viluco,Premier League,37,-71.159347,-34.896282,Chile
2,John Stones,3600000000,5,1994-05-28,Manchester City,Innenverteidiger,"1,88 m",Barnsley,Premier League,26,-1.482776,53.552772,United Kingdom
3,Eric García,630000000,50,2001-01-09,Manchester City,Innenverteidiger,"1,83 m",Barcelona,Premier League,20,2.177432,41.382894,España
4,Joel Latibeaudiere,100000,-,2000-01-06,Man City U23,Innenverteidiger,"1,80 m",Doncaster,Premier League,21,-1.133531,53.522768,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20909,Marko Stojanovic,75000,20,1994-05-31,RW Koblenz,Linksaußen,Linksaußen,Deutschland,Rheinlandpokal,26,10.423447,51.083420,Deutschland
20910,Alen Muharemi,50000,11,1999-07-05,RW Koblenz,Rechtsaußen,Rechtsaußen,Deutschland,Rheinlandpokal,21,10.423447,51.083420,Deutschland
20911,Marko Stojanovic,75000,20,1994-05-31,RW Koblenz,Linksaußen,Linksaußen,Deutschland,Rheinlandpokal,26,10.423447,51.083420,Deutschland
20912,Alen Muharemi,50000,11,1999-07-05,RW Koblenz,Rechtsaußen,Rechtsaußen,Deutschland,Rheinlandpokal,21,10.423447,51.083420,Deutschland


Als nächstes wird die Spalte Size bereinigt. Hier werden ebendso alle Falsch zugewiesenen Daten durch NaN ersetzt, 
da für diesen Spiele dann auf der Website keine Größe vorhanden war.
Desweiteren wird die 'm' angabe entfernt so wie das Kommater.

In [89]:
for index, row in df.iterrows():
    if '30.' in row['Size'] or '31.' in row['Size'] :
        print(row['Size'])
        df.at[index , 'Size'] = np.nan
    if row['Position'] == row['Size'] :
        print(row['Size'])
        df.at[index , 'Size'] = np.nan
        
df['Size'] = df['Size'].str.replace(',', '')
df['Size'] = df['Size'].str.replace('m', '')
df.head(10)

Rechtsaußen                            
Rechter Verteidiger                            
Rechter Verteidiger                            
Defensives Mittelfeld                            
Mittelstürmer                            
Zentrales Mittelfeld                            
Defensives Mittelfeld                            
Mittelstürmer                            
Mittelstürmer                            
Rechter Verteidiger                            
Linker Verteidiger                            
Torwart                            
Defensives Mittelfeld                            
Mittelstürmer                            
Zentrales Mittelfeld                            
Defensives Mittelfeld                            
Linker Verteidiger                            
Innenverteidiger                            
Rechtsaußen                            
Mittelstürmer                            
Linker Verteidiger                            
Mittelstürmer                            
30.0

30.06.2022
Linker Verteidiger                            
Zentrales Mittelfeld                            
30.06.2023
Rechtsaußen                            
Mittelstürmer                            
Rechter Verteidiger                            
Linksaußen                            
Zentrales Mittelfeld                            
31.12.2020
Mittelstürmer                            
Innenverteidiger                            
Zentrales Mittelfeld                            
31.12.2021
Torwart                            
Rechter Verteidiger                            
Linksaußen                            
Torwart                            
Innenverteidiger                            
Innenverteidiger                            
Defensives Mittelfeld                            
Rechtes Mittelfeld                            
Rechtsaußen                            
Mittelstürmer                            
Zentrales Mittelfeld                            
Zentrales Mittelfeld         

Mittelstürmer                            
30.06.2022
Innenverteidiger                            
Offensives Mittelfeld                            
30.06.2020
Mittelstürmer                            
Linker Verteidiger                            
Linker Verteidiger                            
Offensives Mittelfeld                            
30.06.2023
Defensives Mittelfeld                            
Zentrales Mittelfeld                            
30.06.2023
Torwart                            
Innenverteidiger                            
30.06.2023
30.06.2023
Defensives Mittelfeld                            
Rechtes Mittelfeld                            
Offensives Mittelfeld                            
Defensives Mittelfeld                            
Offensives Mittelfeld                            
Zentrales Mittelfeld                            
Rechtes Mittelfeld                            
Rechtes Mittelfeld                            
30.06.2023
Offensives Mittelfeld         

Linker Verteidiger                            
Rechtsaußen                            
Torwart                            
Linkes Mittelfeld                            
Mittelstürmer                            
Innenverteidiger                            
Rechtes Mittelfeld                            
Offensives Mittelfeld                            
Rechtsaußen                            
Torwart                            
Innenverteidiger                            
Defensives Mittelfeld                            
Zentrales Mittelfeld                            
Innenverteidiger                            
Linker Verteidiger                            
Zentrales Mittelfeld                            
Rechtes Mittelfeld                            
Offensives Mittelfeld                            
Torwart                            
Innenverteidiger                            
Innenverteidiger                            
Linker Verteidiger                            
Rechter Verteidi

30.06.2020
30.06.2020
Hängende Spitze                            
30.06.2020
Rechtsaußen                            
30.06.2021
30.06.2020
30.06.2020
30.06.2020
Mittelfeld                            
30.06.2020
30.06.2025
30.06.2022
30.06.2023
30.06.2022
30.06.2022
30.06.2020
Innenverteidiger                            
30.06.2020
30.06.2020
Innenverteidiger                            
30.06.2020
Mittelstürmer                            
30.06.2020
30.06.2020
30.06.2020
30.06.2024
30.06.2020
30.06.2020
Rechter Verteidiger                            
30.06.2020
30.06.2020
Mittelstürmer                            
Rechter Verteidiger                            
30.06.2020
Zentrales Mittelfeld                            
Torwart                            
Innenverteidiger                            
30.06.2020
30.06.2020
30.06.2020
30.06.2020
30.06.2020
30.06.2020
30.06.2020
30.06.2020
Innenverteidiger                            
30.06.2020
30.06.2020
30.06.2020
30.06.2020
Mittelfeld    

Rechter Verteidiger                            
Innenverteidiger                            
Innenverteidiger                            
Mittelstürmer                            
Defensives Mittelfeld                            
Zentrales Mittelfeld                            
Zentrales Mittelfeld                            
Rechter Verteidiger                            
Zentrales Mittelfeld                            
Linkes Mittelfeld                            
Innenverteidiger                            
30.06.2022
Offensives Mittelfeld                            
30.06.2021
Rechter Verteidiger                            
30.06.2020
Defensives Mittelfeld                            
Zentrales Mittelfeld                            
Rechtes Mittelfeld                            
30.06.2024
30.06.2024
30.06.2022
Torwart                            
Innenverteidiger                            
Rechter Verteidiger                            
Rechter Verteidiger                          

30.06.2023
30.06.2022
30.06.2020
31.12.2022
30.06.2021
30.06.2020
Innenverteidiger                            
30.06.2020
30.06.2020
Innenverteidiger                            
30.06.2020
Mittelstürmer                            
30.06.2020
30.06.2024
30.06.2023
30.06.2024
Defensives Mittelfeld                            
30.06.2024
30.06.2020
30.06.2020
30.06.2024
Torwart                            
30.06.2022
30.06.2022
30.06.2023
30.06.2024
30.06.2024
Torwart                            
30.06.2022
Mittelstürmer                            
Mittelstürmer                            
30.06.2021
Linker Verteidiger                            
Defensives Mittelfeld                            
30.06.2020
Defensives Mittelfeld                            
Rechter Verteidiger                            
Innenverteidiger                            
Rechter Verteidiger                            
Defensives Mittelfeld                            
Zentrales Mittelfeld                            


Linksaußen                            
Zentrales Mittelfeld                            
Mittelstürmer                            
Zentrales Mittelfeld                            
30.06.2021
Linksaußen                            
Innenverteidiger                            
Torwart                            
Rechtes Mittelfeld                            
Rechter Verteidiger                            
Zentrales Mittelfeld                            
Rechter Verteidiger                            
Defensives Mittelfeld                            
Zentrales Mittelfeld                            
Offensives Mittelfeld                            
Mittelstürmer                            
Mittelstürmer                            
Mittelstürmer                            
Zentrales Mittelfeld                            
Zentrales Mittelfeld                            
Zentrales Mittelfeld                            
Zentrales Mittelfeld                            
Mittelstürmer              

,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge,Age,Lon,Lat,country
0,Ederson,5600000000,31,1993-08-17,Manchester City,Torwart,188,Osasco (SP),Premier League,27,-46.791680,-23.532486,Brasil
1,Claudio Bravo,120000000,1,1983-04-13,Manchester City,Torwart,184,Viluco,Premier League,37,-71.159347,-34.896282,Chile
2,John Stones,3600000000,5,1994-05-28,Manchester City,Innenverteidiger,188,Barnsley,Premier League,26,-1.482776,53.552772,United Kingdom
3,Eric García,630000000,50,2001-01-09,Manchester City,Innenverteidiger,183,Barcelona,Premier League,20,2.177432,41.382894,España
4,Joel Latibeaudiere,100000,-,2000-01-06,Man City U23,Innenverteidiger,180,Doncaster,Premier League,21,-1.133531,53.522768,United Kingdom
5,Oleksandr Zinchenko,2400000000,11,1996-12-15,Manchester City,Linker Verteidiger,175,"Radomyshl, Zhytomyr ...",Premier League,24,29.228959,50.496181,Україна
6,João Cancelo,3600000000,27,1994-05-27,Manchester City,Rechter Verteidiger,182,Barreiro,Premier League,26,-9.047857,38.634066,Portugal
7,Fernandinho,800000000,25,1985-05-04,Manchester City,Defensives Mittelfeld,179,Londrina,Premier League,35,-51.159502,-23.311288,Brasil
8,Phil Foden,2700000000,47,2000-05-28,Manchester City,Zentrales Mittelfeld,171,Stockport,Premier League,20,-2.160243,53.407901,United Kingdom
9,David Silva,1200000000,21,1986-01-08,Manchester City,Offensives Mittelfeld,170,Arguineguín,Premier League,35,-15.681350,27.759075,España


Da nun bereits die Meisten

In [ ]:
for index, row in df.iterrows(): 
    try:
        if len(row['Size']) != 4 :
            df.at[index , 'Size'] = np.nan
    except: 
        df.at[index , 'Size'] = np.nan
        
df['Size']= df['Size'].astype('float')
df.dtypes

In [4]:
df.drop_duplicates(subset="Players", keep="first")

,Unnamed: 0,Players,Values,Number,BDay,Club,Position,Size,Birth Place,Leauge,Age,Lon,Lat,country
0,0,Ederson,56000000,31,1993-08-17,Manchester City,Torwart,188.0,Osasco (SP),Premier League,27,-46.791680,-23.532486,Brasil
1,1,Claudio Bravo,1200000,1,1983-04-13,Manchester City,Torwart,184.0,Viluco,Premier League,37,-71.159347,-34.896282,Chile
2,2,John Stones,36000000,5,1994-05-28,Manchester City,Innenverteidiger,188.0,Barnsley,Premier League,26,-1.482776,53.552772,United Kingdom
3,3,Eric García,6300000,50,2001-01-09,Manchester City,Innenverteidiger,183.0,Barcelona,Premier League,20,2.177432,41.382894,España
4,4,Joel Latibeaudiere,100000,-,2000-01-06,Man City U23,Innenverteidiger,180.0,Doncaster,Premier League,21,-1.133531,53.522768,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20901,20901,Dimitrios Diamantakos,600000,18,1993-03-05,FC St. Pauli,Mittelstürmer,182.0,Piräus,LOTTO-Pokal,27,23.647059,37.943159,Ελλάδα
20902,20902,Ersen Asani,25000,13,1993-05-28,Altona 93,Innenverteidiger,188.0,Tetovo,LOTTO-Pokal,27,20.971140,42.009970,Македонија
20903,20903,Erdogan Pini,50000,7,1992-01-06,Altona 93,Mittelstürmer,184.0,Hamburg,LOTTO-Pokal,29,10.009913,53.543764,Deutschland
20904,20904,Alen Muharemi,50000,11,1999-07-05,RW Koblenz,Rechtsaußen,NaN,Deutschland,Rheinlandpokal,21,10.423447,51.083420,Deutschland
